# Stability control by local sensitivity analysis

In this notebook, we show how sensitivity analysis can be done with the methods proposed in the main paper.
All the analysis and dynamics functions are defined in the file `gradients.py`.
We also assume prior knowledge of equilibrium points and parameters, contained in the `.pckl` files.

## Calculation of gradients

We start by importing all necessary information:

In [5]:
import gradients as g
import pickle
from pprint import pprint #just for nice print thingies

eqs = pickle.load(open('equilibria_names.pckl', 'rb'))
pars = pickle.load(open('nom_pars.pckl', 'rb'))

# eqs is a dictionary of equilibrium points
pprint(eqs)
print('\n')
# pars is a dictionary of parameters
pprint(pars)

{'ei2D': array([0.35105936, 0.25852722, 2.03117888, 0.58508338]),
 'epi': array([0.12422359, 0.91908678, 1.99746744, 0.26831825]),
 'mes': array([0.83480301, 0.12957192, 1.90140876, 0.1354122 ]),
 'sM': array([0.35144822, 0.25830253, 1.78161153, 0.16158419]),
 'sM2': array([0.35086481, 0.25863976, 2.16901978, 1.17639098]),
 'sS': array([0.12422684, 0.91900697, 2.01793498, 0.29901225]),
 'sen': array([0.12427307, 0.91784816, 2.34692453, 1.56395926])}


{'a1': 2.000000812538125,
 'a10': 0.1018496036626197,
 'a11': 1.0121863302747556,
 'a2': 0.09873700853051456,
 'a3': 2.0001967129905602,
 'a4': 0.0999859413457263,
 'a5': 0.4999709478994542,
 'a6': 0.5001311861499895,
 'a7': 0.5000723533030483,
 'a8': 0.5000870384120284,
 'a9': 2.0062108619945156,
 'be': 0.09939260623322704,
 'bn': 1.0000744796065282,
 'bp': 0.11302922890326285,
 'bs': 0.0906287255831638,
 'k1': 0.09969993769513218,
 'k10': 0.4999672816898334,
 'k11': 0.4999957904362805,
 'k12': 0.4999983899717048,
 'k13': 1.0006202618536

The `gradients.py` file has pre-made functions that serve to calculate local sensitivity to parameter variation, gradients to increase distance between two equilibrium points, and gradients to reduce $R(\lambda)$.

In [7]:
# local sensitivity
sens = g.state_sens(g.F_s, eqs['epi'], pars)
pprint(sens) # prints an nxm array of the sensitivity of the given equilibrium point due to the changes of the DIFFERENTIABLE parameters given in g.df_parameters

# distance gradient
d_gradient = g.distance_gradient(g.F_s, eqs['epi'], eqs['sS'], pars)
pprint(d_gradient) # prints an m-array indicating the direction in differentiable-parameter-space to maximally increase the distance between the given equilibria

# eigenvalue gradient
e, e_gradient = g.eigenvar_gradient(g.F_s, eqs['epi'], pars)
pprint(e_gradient) # prints n-long array where each entry is the direction in parameter space corresponding to the biggest decrease in the corresponding entry in e

array([[ 6.69759691e-01, -1.18278236e-01, -1.23903302e-02,
        -7.50359961e-04, -4.31499158e-01,  2.01148735e-04,
         1.91595622e-03, -1.21659259e-04, -2.07087918e-05,
        -6.66774615e-05, -2.09929839e-05, -6.56814337e-05,
         1.31463825e-05, -4.03759590e-05, -2.44810042e-05,
        -7.25721320e-04,  1.68901833e-02,  1.51993754e-01,
        -1.87307942e-02, -2.43661724e-03,  1.09948515e-05,
         1.45953265e-04,  1.77990870e-04,  4.22907080e-05,
         2.15994096e-04,  4.80120136e-04,  3.79801273e-04,
         1.45223956e+00, -4.60036215e-02,  1.89143338e-04,
         2.79047217e-03],
       [-5.22240366e+00,  9.22266149e-01,  9.66127203e-02,
         5.85087855e-03,  1.28146895e+01, -5.97372797e-03,
        -5.69001902e-02,  3.61304443e-03,  5.08116622e-04,
         1.63601657e-03,  5.15089638e-04,  1.61157775e-03,
        -3.22563264e-04,  9.90675655e-04,  6.00672665e-04,
         1.78064983e-02, -1.31700006e-01, -1.18516051e+00,
         5.56268320e-01,  7.23

In order to obtain a new parameter set after moving in the desired direction and calculating the new equilibrium points and a brief summary of wether or not there were bifurcations, the function `g.control()` was made.

In [8]:
out = g.control([d_gradient], 1e-3, pars, eqs)
pprint(out)

{'lost': [],
 'out': ({'a1': 2.0000048330639477,
          'a10': 0.10171180941382706,
          'a11': 1.0120656238856178,
          'a2': 0.09877318469750197,
          'a3': 2.00018132142272,
          'a4': 0.09998394878423925,
          'a5': 0.4999705379853026,
          'a6': 0.500125744427395,
          'a7': 0.5000657211183622,
          'a8': 0.5000856001814815,
          'a9': 2.0061429670143474,
          'be': 0.0993548033417071,
          'bn': 1.0000674277228045,
          'bp': 0.11223245608143527,
          'bs': 0.0909743865676118,
          'k1': 0.09985936660274013,
          'k10': 0.4999697675285229,
          'k11': 0.4999965800107825,
          'k12': 0.5000006966633096,
          'k13': 1.0006161293384148,
          'k14': 1.9998537410237751,
          'k15': 1.9999250234294936,
          'k16': 0.9866870876165653,
          'k2': 0.1010160394757167,
          'k3': 2.000096689507297,
          'k4': 0.09999530963714127,
          'k5': 0.10259322616837857,
   

The output of this dictionary is:
- `lost`: a list of the equilibria lost after parameter manipulation, if any.
- `out`: a tuple containing a new dictionary of parameters, and its corresponding dictionary of equilibrium points.
- `quit`: a boolean indicating if iterations should be stopped due to a bifurcation.

## Multiple gradient descent
In order to calculate the optimum direction of movement, the function `g.optimum_coefficients()` was made.
This calculates the optimum coefficients as described by LeNovére, and returns the direction of descent.

In [10]:
opt = g.optimum_coefficients(e_gradient)
pprint(opt)

array([ 2.85975964e-02, -1.50578314e-02,  3.54320731e-03, -2.00797535e-03,
        4.00202153e-02,  2.27117399e-04,  3.16144991e-03, -2.95460671e-04,
       -3.90795204e-03, -1.13217214e-02, -8.82373707e-02, -2.26331526e-02,
       -1.45725105e-04, -4.22298042e-03,  5.41767335e-05,  1.66848444e-02,
        7.26567347e-04, -4.41914415e-02,  1.85315766e-03, -4.04304933e-03,
        2.08976161e-03,  2.44247785e-02, -1.80943238e-02,  1.75922358e-02,
       -2.17000850e-03,  5.02174689e-02, -9.35413139e-03,  2.31216719e-02,
        4.13684068e-03,  3.17897158e-02, -6.97937263e-03])


Note that this function is called by `g.control()`, so passing an `array` or `list` of gradients can be done (in fact, even if it is only one gradient, it should be put in such an iterable).
So, in order to move along the direction that reduces **all** eigenvalues of the equilibrium *epi* (gradients calculated above), we would only need to call

In [11]:
out = g.control(e_gradient, 1e-3, pars, eqs)
pprint(out)

{'lost': [],
 'out': ({'a1': 2.000006105001265,
          'a10': 0.10221539783742793,
          'a11': 1.0121181928945173,
          'a2': 0.09841510915477204,
          'a3': 2.0002102117647937,
          'a4': 0.0999564909587359,
          'a5': 0.4999861701446301,
          'a6': 0.5003091011638942,
          'a7': 0.499940550597855,
          'a8': 0.5002151838068079,
          'a9': 2.0061950552148136,
          'be': 0.09942273981532128,
          'bn': 1.000306042309953,
          'bp': 0.11297838974497101,
          'bs': 0.09079714850582603,
          'k1': 0.09990824835666354,
          'k10': 0.4998848119873265,
          'k11': 0.49935305162633925,
          'k12': 0.49983352552177684,
          'k13': 1.0006192003626329,
          'k14': 1.9998113919215201,
          'k15': 1.9999184287168839,
          'k16': 0.9865800957771433,
          'k2': 0.10093450626814576,
          'k3': 2.000125448021165,
          'k4': 0.09998085997826638,
          'k5': 0.1032393188236346,


## Iterative manipulation until goals are reached
The code to manipulate the system parameters until one or several control conditions are reached could look something like the following:

```python
import gradients as g
import pickle
import other_packages

eqs = pickle.load(...) # reference points
pars = pickle.load(...) # nominal parameters
new_eqs = eqs.copy()
new_pars = pars.copy()

### Define control conditions
step_size = 1e-3
n_iter = 1e3
e_target = 1 # eigenvalue change target
eigenval_diff = # calcualate eigenval difference
q = False

ii = 0
while ii < n_iter and eigenval_diff < e_target and not q:
    e, d = g.eigenvar_gradient(g.F_s, new_eqs['epi'], new_pars)
    lost, new, q = g.control(d, step_size, new_pars, new_eqs)
    
    eigenval_diff = # calculate eigenval difference again

print('This is done. The control targets have been reached.')
```

In the end, parameter values, equilibrium points or anything similar could be saved by whatever means.
Plots such as those shown in the main paper can be done, but would need to be done manually as such functions have not been done.
The code used to generate such figures is not shown here as it is very slow due to repeated symbolic operations and substitutions.